In [1]:
# Install the Ultralytics library, which contains YOLOv8
%pip install ultralytics


Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install roboflow

In [3]:
from ultralytics import YOLO
import torch
from pathlib import Path
import matplotlib.pyplot as plt


In [6]:
from roboflow import Roboflow
import os
from dotenv import load_dotenv

api_key = os.getenv('ROBOFLOW_API_KEY')
rf = Roboflow(api_key)                # your key

ws = rf.workspace("cargosnap")       # make sure this slug matches the URL

print("Projects in workspace:")
for slug in ws.projects():           # returns strings
    print("  •", slug)

loading Roboflow workspace...
Projects in workspace:
  • cargosnap/container-character-detection
  • cargosnap/unified-detection-0zmvz
  • cargosnap/vertical-code-area-detection
  • cargosnap/vertical-ocr


In [7]:
# Access the workspace and project
project = rf.workspace("cargosnap").project("unified-detection-0zmvz")

# Get version x and download the dataset in YOLO format (or other, see below)
dataset = project.version(7).download("yolov8")  # or "yolov8", "coco", etc.

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to Unified-detection-7 in yolov8:: 100%|██████████| 2647/2647 [00:00<00:00, 8806.45it/s]


In [ ]:
# If restarted in the mean while, be sure to reload "from ultralytics import YOLO"
# Load a pre-trained YOLOv8n model
model = YOLO("yolov8n.pt")

# Train the model
# Note: The first time you run this, it will download the yolov8n.pt weights.
results = model.train(
    data="Unified-detection-7/data.yaml",
    epochs=150,  # As per the plan, start with 50 and increase if needed
    imgsz=640,  # Standard input size for YOLO
    device="cuda",
    project="runs/detect",
    name="detection_exp"  # Name for the experiment directory
)

# files.download('runs/detect/detection_exp2/weights/best_saved_model/best_int8.tflite')

In [ ]:
# optional: slightly tuned parameter "cosine LR scheduler"
# Load a pre-trained YOLOv8n model
# model = YOLO("yolov8n.pt")

# Train the model
model.train(
    data="Unified-detection-4/data.yaml",
    epochs=100,
    imgsz=640,
    device="cuda",
    project="runs/detect",
    name="detection_exp_cosine",
    lr_scheduler="cosine"
)

In [ ]:
from ultralytics.utils.plotting import plot_results

# Compare run 1 and run 2
plot_results(
    files=[
        "runs/detect/detection_exp/results.csv",
        "runs/detect/detection_exp_cosine/results.csv"
    ],
    labels=["baseline", "cosine"]
)

In [2]:
from ultralytics import YOLO

# files.download("/best-OCR-Colab-22-06-25.pt")

# Load best weights
model = YOLO("../models/unified-detection-v7.pt")

# Run evaluation on test set (ensure YAML defines 'test' section properly)
results = model.val(data="Unified-detection-7/data.yaml", split="test")

# Extract and print mAP results
print("mAP@0.5:", results.results_dict.get("metrics/mAP50(B)", "N/A"))
print("mAP@0.5:0.95:", results.results_dict.get("metrics/mAP50-95(B)", "N/A"))

# results.plot_confusion_matrix(normalize=True, save_dir="conf_matrix")

Ultralytics 8.3.154 🚀 Python-3.9.6 torch-2.3.0 CPU (Apple M4)
Model summary (fused): 72 layers, 3,006,623 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 142.9±59.4 MB/s, size: 31.4 KB)


val: Scanning /Users/marcelmerkx/Development/universal-scanner/detection/notebooks/Unified-detection-7/test/labels... 65 images, 1 backgrounds, 0 corrupt: 100%|██████████| 65/65 [00:00<00:00, 3505.73it/s]

val: New cache created: /Users/marcelmerkx/Development/universal-scanner/detection/notebooks/Unified-detection-7/test/labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:08<00:00,  1.71s/it]


                   all         65         66      0.949      0.918      0.954      0.675
      code_container_h         16         16      0.926          1      0.995      0.737
      code_container_v          9          9      0.892          1      0.973      0.505
    code_license_plate         15         15      0.924          1      0.958      0.744
       code_qr_barcode         10         12          1      0.816      0.977      0.703
             code_seal         14         14          1      0.771      0.867      0.684
Speed: 1.0ms preprocess, 128.0ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to runs/detect/val
mAP@0.5: 0.953862852833441
mAP@0.5:0.95: 0.674519443018419


In [ ]:
from ultralytics import YOLO

# Load your trained model (upload the .pt file to Colab first)
model = YOLO("/content/unified-detection-v7.pt")

# Export with NNAPI-compatible settings
model.export(
    format="onnx",
    imgsz=640,
    half=False,        # ✅ FP32 instead of FP16 
    simplify=True,     # ✅ Graph optimization
    opset=11,          # ✅ Better NNAPI support
    dynamic=False      # ✅ Static shapes
)

In [ ]:
# testing TFLite export:
from ultralytics import YOLO

# Load best weights
model = YOLO("/content/unified-detection-v7.pt")

model.export(
    format="tflite",
    imgsz=416,  # Smaller size for mobile
    int8=False,
    data=None,  # No quantization dataset yet
    verbose=True
)

model.export(
    format="tflite", 
    imgsz=416,
    half=True,  # FP16 quantization
    verbose=True
)

model.export(
    format="tflite",
    imgsz=416,
    int8=True,  # Full integer quantization
    verbose=True
)

In [ ]:
!zip -r runs.zip runs/

In [ ]:
from google.colab import files   # already available in Colab

# pick the ones you need
#files.download('runs/detect/container_exp/weights/best_saved_model/best_int8.tflite')
#files.download('runs/detect/container_exp/weights/best.mlpackage')
files.download('runs/detect/container_exp/weights/best.onnx')

In [ ]:
from ultralytics import YOLO

# model = YOLO("/Users/marcelmerkx/Development/container-digit-detector/models/best-YOLO-10-06-25.pt")
model = YOLO("runs/detect/container_exp/weights/best.pt")
results = model.predict(
    source="Container-Character-OCR-1/test/images/ACLU9698632_jpg.rf.8aa31ec02a1dce12043ab484d47e10fc.jpg",
    conf=0.1,         # set confidence threshold
    save=True,
)


image 1/1 /content/Container-Character-OCR-1/test/images/ACLU9698632_jpg.rf.8aa31ec02a1dce12043ab484d47e10fc.jpg: 640x128 1 2, 1 3, 2 6s, 1 8, 2 9s, 1 A, 1 C, 1 L, 1 U, 39.0ms
Speed: 1.3ms preprocess, 39.0ms inference, 1.9ms postprocess per image at shape (1, 3, 640, 128)
Results saved to runs/detect/predict


In [ ]:
# not used (but pretty good:)

import os
import logging
from pathlib import Path
from ultralytics import YOLO

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[
        logging.StreamHandler(),
        logging.FileHandler("training.log")
    ]
)
logger = logging.getLogger(__name__)

def train_model(
    data_yaml: str = "YOLO-OCR/container_dataset.yaml",
    epochs: int = 150,
    batch_size: int = 16,
    img_size: int = 640,
    device: str = "cuda"
) -> None:
    """
    Train a YOLOv8 model on the annotated dataset.

    Args:
        data_yaml: Path to the dataset configuration file
        epochs: Number of training epochs
        batch_size: Batch size for training
        img_size: Input image size
        device: Device to use for training ('cpu' for CPU training)
    """
    try:
        # Initialize model
        logger.info("Initializing YOLOv8 model...")
        model = YOLO("yolov8n.pt")  # Load the smallest YOLOv8 model

        # Create results directory
        results_dir = Path("results")
        results_dir.mkdir(exist_ok=True)

        # Training configuration
        logger.info("Starting training with configuration:")
        logger.info(f"- Epochs: {epochs}")
        logger.info(f"- Batch size: {batch_size}")
        logger.info(f"- Image size: {img_size}")
        logger.info(f"- Device: {device}")

        # Train the model
        results = model.train(
            data=data_yaml,
            epochs=epochs,
            batch=batch_size,
            imgsz=img_size,
            device=device,
            project="results",
            name="train",
            exist_ok=True,
            pretrained=True,
            optimizer="auto",
            verbose=True,
            seed=42,
            deterministic=True,
            plots=True,  # Generate training plots
            save=True,  # Save best model
            save_period=10,  # Save checkpoint every 10 epochs
        )

        logger.info("Training completed successfully!")
        logger.info(f"Results saved in: {results_dir / 'train'}")

        # Print final metrics
        metrics = results.results_dict
        logger.info("\nFinal metrics:")
        logger.info(f"mAP50: {metrics.get('metrics/mAP50(B)', 'N/A')}")
        logger.info(f"mAP50-95: {metrics.get('metrics/mAP50-95(B)', 'N/A')}")
        logger.info(f"Precision: {metrics.get('metrics/precision(B)', 'N/A')}")
        logger.info(f"Recall: {metrics.get('metrics/recall(B)', 'N/A')}")

    except Exception as e:
        logger.error(f"An error occurred during training: {str(e)}")
        raise

if __name__ == "__main__":
    train_model()

Ultralytics 8.3.154 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=YOLO-OCR/container_dataset.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained

100%|██████████| 755k/755k [00:00<00:00, 97.1MB/s]

Overriding model.yaml nc=80 with nc=36

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytic

Model summary: 129 layers, 3,017,868 parameters, 3,017,852 gradients, 8.2 GFLOPs

Transferred 319/355 items from pretrained weights
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...


100%|██████████| 5.35M/5.35M [00:00<00:00, 350MB/s]


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    100/100      4.06G     0.6233     0.7004     0.8144        139        640: 100%|██████████| 18/18 [00:04<00:00,  4.50it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  3.66it/s]


                   all         50        528      0.815      0.694      0.762      0.606

100 epochs completed in 0.154 hours.
Optimizer stripped from results/train/weights/last.pt, 6.2MB
Optimizer stripped from results/train/weights/best.pt, 6.2MB

Validating results/train/weights/best.pt...
Ultralytics 8.3.154 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,012,668 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  3.01it/s]


                   all         50        528      0.828      0.714      0.776      0.625
                     A         12         12      0.897      0.725      0.891      0.674
                     B          8          9      0.875      0.777      0.895      0.702
                     C         14         14      0.706       0.86      0.844      0.756
                     D          9          9      0.768      0.556      0.839      0.567
                     E          9          9      0.724      0.889      0.808      0.654
                     F          4          5       0.92        0.6      0.928      0.802
                     G          5          5          1        0.4      0.768      0.623
                     H          3          3      0.554          1      0.641       0.53
                     I          6          6      0.648      0.667       0.75      0.552
                     J          1          1          1          0     0.0158     0.0142
                     

In [ ]:
# Load your custom-trained model from the saved weights
model_path = Path("/Users/marcelmerkx/Development/container-digit-detector/models/best-YOLO-18-06-25.pt")

if not model_path.exists():
    print(f"Model weights not found at {model_path}.")
    print("Please ensure the model has been trained successfully.")
else:
    model = YOLO(model_path)

    # --- IMPORTANT ---
    # Replace this with a path to a REAL image from your validation set.
    # For example: test_image_path = "data/dataset/images/val/your_image_name.jpg"
    test_image_path = "../data/dataset/images/val/ACLU9689902.jpg"

    if not Path(test_image_path).exists():
        print(f"Test image not found at {test_image_path}")
        print("Please update the `test_image_path` variable with a valid file path.")
    else:
        # Run inference on the test image
        inference_results = model(test_image_path)

        # The results object contains bounding boxes, confidence scores, and classes.
        # We can use the .plot() method to easily visualize the results.
        for r in inference_results:
            im_array = r.plot()  # This plots predictions on the image

            # Display the image with predictions using Matplotlib
            plt.figure(figsize=(12, 12))
            plt.imshow(im_array[..., ::-1])  # Convert BGR to RGB for correct color
            plt.title("Inference Result")
            plt.axis("off")
            plt.show()
